# Conversational AI (Shopstore Assistant) - aka Chatbot!

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

# Initialize
openai = OpenAI()
MODEL = 'gpt-4.1-mini'

In [ ]:

system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

There's an important concept here. Notice how one manipulates the system prompt with relevant information based on the user's input? Once manipulate the system prompt to do things like:
- summarize (to avoid token cost)
- change the nature or speciailized information available to the LLM based on the questions you ask.

It's your first glimpse into what RAG is about. Although this isn't RAG, its introducing a problem that RAG is well suited to solve.

#### Scrubbing the history.
```python
history = [{"role":h["role"], "content":h["content"]} for h in history]
```
This line above is a scrubber to make sure that your history is clean. Its not necessary for OpenAI but it is important for Gemini, Grok, and a few others. This code looks at every conversation in the history coming back from the Gradio interface and pulls out the role and the content only. OpenAI throws away the missing attributes but some other APIs will give you issues if you don't do this.

#### Assembly the full history
```python
messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]
```
Gradio has no idea what the system message is, so you need to add the "relevant system message" in before the history and the current message. The line above simply takes care of that.


In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()